<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/three_plus_word_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove posts with only 1 or 2 words. Leave response length at 20.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "three_plus_words"
MODEL_NAME = "three_plus_words"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=28c043e7bd383a894646d4e5089d1a4523cca3068527dc2317459a65410d662e
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
len(dev_df)

2292907

In [14]:
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
train_df['split_response'] = train_df['response_text'].str.split()
train_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican,"[protecting, birth, is, not, the, same, as, protecting, life., you, may, very, well, pledge, to,..."
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican,"[you, need, to, protect, children, and, leave, my, body, to, me.]"
2,2,57265377,M,0,Sherry,thank you,Roger Williams,Congress_Republican,"[thank, you]"
3,3,57265377,M,0,Bob,thank you roger,Roger Williams,Congress_Republican,"[thank, you, roger]"
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican,"[unwanted, pregnancy, is, a, sad, and, unfortunate, situation, for, anyone, to, find, themselves..."


In [15]:
len(train_df)

9879016

In [19]:
# Remove the split responses that have length 1 or 2.
mask = (train_df['split_response'].str.len() > 2) 
train_df = train_df.loc[mask]
len(train_df)

8685575

In [20]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
mask2 = (dev_df['split_response'].str.len() > 2) 
dev_df = dev_df.loc[mask2]
len(dev_df)

2041945

In [0]:
len(dev_df)

390738

In [21]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3721104, 9)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [23]:
train_df.op_gender.value_counts()

M    2725461
W     995643
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do oversampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using oversampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [28]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 51 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [30]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
\ [4 files][  2.1 GiB/  2.1 GiB]   38.0 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
-
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [32]:
embedding_matrix.shape

(547237, 100)

In [33]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           54723700  
_________________________________________________________________
conv1d (Conv1D)              (None, 46, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [37]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3721104 samples, validate on 2000000 samples
Epoch 1/30
3721104/3721104 [==============================] - 222s 60us/sample - loss: 0.4935 - acc: 0.7774 - val_loss: 0.4240 - val_acc: 0.8466
Epoch 2/30
3721104/3721104 [==============================] - 206s 55us/sample - loss: 0.4748 - acc: 0.7860 - val_loss: 0.4287 - val_acc: 0.8424
Epoch 3/30
3721104/3721104 [==============================] - 207s 56us/sample - loss: 0.4693 - acc: 0.7883 - val_loss: 0.4378 - val_acc: 0.8373
Epoch 4/30
3721104/3721104 [==============================] - 212s 57us/sample - loss: 0.4660 - acc: 0.7896 - val_loss: 0.4266 - val_acc: 0.8415
Epoch 5/30
3721104/3721104 [==============================] - 224s 60us/sample - loss: 0.4635 - acc: 0.7906 - val_loss: 0.4319 - val_acc: 0.8412
Epoch 6/30
3721104/3721104 [==============================] - 230s 62us/sample - loss: 0.4617 - acc: 0.7913 - val_loss: 0.4413 - val_acc: 0.8363
Epoch 7/30
3721104/3721104 [==============================] - 230s 62us/samp

In [38]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7986
Testing Accuracy:  0.8327


In [39]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6385719026713498


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [44]:
wrong_preds.op_gender.value_counts()

W    271147
M     70449
Name: op_gender, dtype: int64

In [45]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.053850128186606394

In [46]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
303581,2568116,59227815,M,63440,Susan,go jackie go jackie russo migliaccio,Paul Ryan,Congress_Republican,"[go, jackie, go, jackie, russo, migliaccio]",0.044657,W
1973690,11023082,86569077,W,389785,Patti,everyone should remember this when it comes time to vote - especially those students of voting age.,Kirsten Gillibrand,Congress_Democratic,"[everyone, should, remember, this, when, it, comes, time, to, vote, -, especially, those, studen...",0.729613,M
2068120,11207874,48870736,W,395569,Matthew,is this really what this congressman is wasting her time with? i might move to minnesota just t...,Betty McCollum,Congress_Democratic,"[is, this, really, what, this, congressman, is, wasting, her, time, with?, i, might, move, to, m...",0.936863,M
1915656,10965048,86569077,W,389341,Stacey,thank you senator. i only hope besides the democrats that their are republicans who realize thi...,Kirsten Gillibrand,Congress_Democratic,"[thank, you, senator., i, only, hope, besides, the, democrats, that, their, are, republicans, wh...",0.734139,M
2021709,11071101,86569077,W,390228,Gary,not really a tax cut it just extends the bush tax cuts. as much as i do not like paying taxes cu...,Kirsten Gillibrand,Congress_Democratic,"[not, really, a, tax, cut, it, just, extends, the, bush, tax, cuts., as, much, as, i, do, not, l...",0.809737,M
2137943,11278923,77234050,W,399911,Gordon,"republicans have a spending problem, but the only time they cry about spending when democrats ar...",Cathy McMorris Rodgers,Congress_Republican,"[republicans, have, a, spending, problem,, but, the, only, time, they, cry, about, spending, whe...",0.722735,M
1925962,10975354,86569077,W,389342,Gary,"sign up for the selective service act. if not, shut up.",Kirsten Gillibrand,Congress_Democratic,"[sign, up, for, the, selective, service, act., if, not,, shut, up.]",0.723485,M
1086539,5335593,46506154,M,161584,Tim,americans should be entitled to the best talent in this country to lead and to recover from tran...,Marco Rubio,Congress_Republican,"[americans, should, be, entitled, to, the, best, talent, in, this, country, to, lead, and, to, r...",0.485492,W
1995396,11044788,86569077,W,390006,Mariatou,travel mercies for your family!,Kirsten Gillibrand,Congress_Democratic,"[travel, mercies, for, your, family!]",0.789316,M
2161028,11302008,77234050,W,400796,Ron,"support. judging by the complaining from both sides, the compromise is just about right. @stev...",Cathy McMorris Rodgers,Congress_Republican,"[support., judging, by, the, complaining, from, both, sides,, the, compromise, is, just, about, ...",0.715549,M


In [47]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
373648,2653944,13876473,W,70257,Donald,keep the pressure on rep maloney. rep issa is a thug and jerk.,Carolyn Maloney,Congress_Democratic,"[keep, the, pressure, on, rep, maloney., rep, issa, is, a, thug, and, jerk.]",0.999977,M
373456,2653752,13876473,W,70137,Beth,thank you rep. maloney - your wisdom in this matter is greatly appreciated byl those who appreci...,Carolyn Maloney,Congress_Democratic,"[thank, you, rep., maloney, -, your, wisdom, in, this, matter, is, greatly, appreciated, byl, th...",0.999977,M
373466,2653762,13876473,W,70137,Frank,"thank you, rep maloney!",Carolyn Maloney,Congress_Democratic,"[thank, you,, rep, maloney!]",0.999973,M
369249,2649545,13876473,W,69492,Hannah,"thank you, rep. maloney.",Carolyn Maloney,Congress_Democratic,"[thank, you,, rep., maloney.]",0.999973,M
373401,2653697,13876473,W,70118,Jen,"yes, thank you, rep. maloney!",Carolyn Maloney,Congress_Democratic,"[yes,, thank, you,, rep., maloney!]",0.999963,M
369109,2649405,13876473,W,69489,Noel,thank you congressman maloney,Carolyn Maloney,Congress_Democratic,"[thank, you, congressman, maloney]",0.999940,M
1951561,11000953,86569077,W,389558,Basil,thank you senator gillibrand. please do all you can to encourage other senators to do the same. ...,Kirsten Gillibrand,Congress_Democratic,"[thank, you, senator, gillibrand., please, do, all, you, can, to, encourage, other, senators, to...",0.999925,M
1951013,11000405,86569077,W,389558,Johanna,thank you senator gillibrand I will take an attempt at peace any day. i will have to call senat...,Kirsten Gillibrand,Congress_Democratic,"[thank, you, senator, gillibrand, I, will, take, an, attempt, at, peace, any, day., i, will, hav...",0.999913,M
373058,2653354,13876473,W,69956,Robert,go rep. maloney!,Carolyn Maloney,Congress_Democratic,"[go, rep., maloney!]",0.999905,M
372789,2653085,13876473,W,69837,Sylvie,"you have my support, rep maloney!",Carolyn Maloney,Congress_Democratic,"[you, have, my, support,, rep, maloney!]",0.999886,M


In [48]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
887235,5136289,46506154,M,160981,Sven,"trash scott brown all you want, but he is infinitely better in the senate than jeanne shaheen. j...",Marco Rubio,Congress_Republican,"[trash, scott, brown, all, you, want,, but, he, is, infinitely, better, in, the, senate, than, j...",0.000005,W
888623,5137677,46506154,M,160997,Adam,lmao at the libs on here!!! to much obama kool aid you drank in the past. here in iowa joni ha...,Marco Rubio,Congress_Republican,"[lmao, at, the, libs, on, here!!!, to, much, obama, kool, aid, you, drank, in, the, past., here,...",0.000014,W
1870185,10903113,10458808,M,385776,Hannah,i see tulsi!,Mark Takano,Congress_Democratic,"[i, see, tulsi!]",0.000039,W
536045,4785099,46506154,M,159974,Jim,"the us chamber of commerce says if you vote for trump, you should elect a democrat for the senat...",Marco Rubio,Congress_Republican,"[the, us, chamber, of, commerce, says, if, you, vote, for, trump,, you, should, elect, a, democr...",0.000042,W
1855248,10888176,10458808,M,385767,Karen,"thank you representative esty, for fighting this fight for years!",Mark Takano,Congress_Democratic,"[thank, you, representative, esty,, for, fighting, this, fight, for, years!]",0.000044,W
226949,2403701,62246450,M,52863,Bryce,"good luck congresswomen suzanne bonamici, we are pulling for you to be a champion for oregon's f...",Ron Wyden,Congress_Democratic,"[good, luck, congresswomen, suzanne, bonamici,, we, are, pulling, for, you, to, be, a, champion,...",0.000052,W
889876,5138930,46506154,M,161005,Laurie,saw them tonight ...they were great! go scott! cannot wait to vote for you in 4 weeks; send shah...,Marco Rubio,Congress_Republican,"[saw, them, tonight, ...they, were, great!, go, scott!, cannot, wait, to, vote, for, you, in, 4,...",0.000052,W
197830,2374582,62246450,M,52667,Darrell,tensions are running high between debbie wasserman schultz and tulsi gabbard. the democratic deb...,Ron Wyden,Congress_Democratic,"[tensions, are, running, high, between, debbie, wasserman, schultz, and, tulsi, gabbard., the, d...",0.000070,W
1887369,10920297,10458808,M,386070,Stephen,yes yes yes. kamala. you have always been there for us!!! fantastic!,Mark Takano,Congress_Democratic,"[yes, yes, yes., kamala., you, have, always, been, there, for, us!!!, fantastic!]",0.000091,W
303931,2568466,59227815,M,63440,Harvetta,how about the emolument clause of the constitution where it saids that no one will accept foreig...,Paul Ryan,Congress_Republican,"[how, about, the, emolument, clause, of, the, constitution, where, it, saids, that, no, one, wil...",0.000100,W
